In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_yinyue_VOCALOID·UTAU=pd.DataFrame(columns=columns)
channel_yinyue_VOCALOID·UTAU

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').click()  #选择“音乐”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'音乐'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[4]/a').click()  #选择“VOCALOID·UTAU”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[4]/a').text
sub_channel

'VOCALOID·UTAU'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [13]:
print(datetime.datetime.now())

for month in range(8,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_yinyue_VOCALOID·UTAU.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-19 09:32:28.959931


<ipython-input-13-b8fa6b974ae8>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-13-b8fa6b974ae8>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-13-b8fa6b974ae8>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1p5411879s视频属性的抓取
BV号为1p5411879s的视频属性抓取结束


<ipython-input-13-b8fa6b974ae8>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Ya4y1E7pk视频属性的抓取
BV号为1Ya4y1E7pk的视频属性抓取结束
开始BV号为1xD4y127im视频属性的抓取
BV号为1xD4y127im的视频属性抓取结束
开始BV号为1Bi4y1M7ro视频属性的抓取
BV号为1Bi4y1M7ro的视频属性抓取结束
开始BV号为1nz4y1Q7UJ视频属性的抓取
BV号为1nz4y1Q7UJ的视频属性抓取结束
开始BV号为1VD4y1m7Px视频属性的抓取
BV号为1VD4y1m7Px的视频属性抓取结束
开始BV号为1Zi4y1M75s视频属性的抓取
BV号为1Zi4y1M75s的视频属性抓取结束
开始BV号为1F54y1U7WE视频属性的抓取
BV号为1F54y1U7WE的视频属性抓取结束
开始BV号为1N54y1U73o视频属性的抓取
BV号为1N54y1U73o的视频属性抓取结束
开始BV号为1Vv411i7UU视频属性的抓取
BV号为1Vv411i7UU的视频属性抓取结束
开始BV号为1hk4y117kk视频属性的抓取
BV号为1hk4y117kk的视频属性抓取结束
开始BV号为1dD4y1U7EQ视频属性的抓取
BV号为1dD4y1U7EQ的视频属性抓取结束
开始BV号为1DC4y1t7YN视频属性的抓取
BV号为1DC4y1t7YN的视频属性抓取结束
开始BV号为1i54y1m7vg视频属性的抓取
BV号为1i54y1m7vg的视频属性抓取结束
开始BV号为16K4y1Y7Qa视频属性的抓取
BV号为16K4y1Y7Qa的视频属性抓取结束
开始BV号为1d5411a7TT视频属性的抓取
BV号为1d5411a7TT的视频属性抓取结束
开始BV号为1jC4y1t7ac视频属性的抓取
BV号为1jC4y1t7ac的视频属性抓取结束
开始BV号为1R64y1F7yh视频属性的抓取
BV号为1R64y1F7yh的视频属性抓取结束
开始BV号为18Z4y1M7wd视频属性的抓取
BV号为18Z4y1M7wd的视频属性抓取结束
开始BV号为1N64y1c7sE视频属性的抓取
BV号为1N64y1c7sE的视频属性抓取结束
开始BV号为1Gt4y1D7Pq视频属性的抓取
BV号为1Gt4y1D7Pq的视频属性抓取结束
开始BV号为1Af4y1X79e视频属性的抓取
开始BV号为1Af4y1X79e

开始BV号为1wz4y1Z7bD视频属性的抓取
BV号为1wz4y1Z7bD的视频属性抓取结束
开始BV号为1H5411b7KY视频属性的抓取
BV号为1H5411b7KY的视频属性抓取结束
开始BV号为1ev411C7Dd视频属性的抓取
BV号为1ev411C7Dd的视频属性抓取结束
开始BV号为1AT4y1A7aY视频属性的抓取
BV号为1AT4y1A7aY的视频属性抓取结束
开始BV号为1jZ4y157X8视频属性的抓取
BV号为1jZ4y157X8的视频属性抓取结束
开始BV号为1yD4y1R7gy视频属性的抓取
BV号为1yD4y1R7gy的视频属性抓取结束
开始BV号为1NK4y1a762视频属性的抓取
BV号为1NK4y1a762的视频属性抓取结束
开始BV号为1vK411K7DP视频属性的抓取
BV号为1vK411K7DP的视频属性抓取结束
开始BV号为1oy4y1C7Pn视频属性的抓取
BV号为1oy4y1C7Pn的视频属性抓取结束
开始BV号为1Lk4y117be视频属性的抓取
BV号为1Lk4y117be的视频属性抓取结束
开始BV号为1kK4y1a7UG视频属性的抓取
BV号为1kK4y1a7UG的视频属性抓取结束
开始BV号为1ZV411m7u1视频属性的抓取
BV号为1ZV411m7u1的视频属性抓取结束
开始BV号为1gv41117P7视频属性的抓取
BV号为1gv41117P7的视频属性抓取结束
开始BV号为1bh411R7DR视频属性的抓取
BV号为1bh411R7DR的视频属性抓取结束
开始BV号为1954y127U5视频属性的抓取
BV号为1954y127U5的视频属性抓取结束
开始BV号为11t4y1S77g视频属性的抓取
BV号为11t4y1S77g的视频属性抓取结束
开始BV号为15i4y1u7yD视频属性的抓取
BV号为15i4y1u7yD的视频属性抓取结束
开始BV号为1uv41117BR视频属性的抓取
BV号为1uv41117BR的视频属性抓取结束
开始BV号为1H54y1m77b视频属性的抓取
BV号为1H54y1m77b的视频属性抓取结束
开始BV号为1a5411b7kQ视频属性的抓取
BV号为1a5411b7kQ的视频属性抓取结束
开始BV号为1DC4y1t7Qq视频属性的抓取
BV号为1DC4y1t7Qq的视

<ipython-input-13-b8fa6b974ae8>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1gA411j7fr视频属性的抓取
BV号为1gA411j7fr的视频属性抓取结束
开始BV号为1gv411C7Cn视频属性的抓取
BV号为1gv411C7Cn的视频属性抓取结束
开始BV号为1Ja411A76q视频属性的抓取
BV号为1Ja411A76q的视频属性抓取结束
开始BV号为1Ti4y177eN视频属性的抓取
BV号为1Ti4y177eN的视频属性抓取结束
开始BV号为1SD4y197tP视频属性的抓取
BV号为1SD4y197tP的视频属性抓取结束
开始BV号为1B54y1R7py视频属性的抓取
BV号为1B54y1R7py的视频属性抓取结束
开始BV号为1YT4y1F7WH视频属性的抓取
BV号为1YT4y1F7WH的视频属性抓取结束
开始BV号为1eK411A75w视频属性的抓取
BV号为1eK411A75w的视频属性抓取结束
开始BV号为15y4y1C7H3视频属性的抓取
BV号为15y4y1C7H3的视频属性抓取结束
开始BV号为1iz4y1C7Vq视频属性的抓取
BV号为1iz4y1C7Vq的视频属性抓取结束
开始BV号为1vf4y1i7Rf视频属性的抓取
BV号为1vf4y1i7Rf的视频属性抓取结束
开始BV号为1A54y1r7up视频属性的抓取
BV号为1A54y1r7up的视频属性抓取结束
开始BV号为1zD4y197Vi视频属性的抓取
BV号为1zD4y197Vi的视频属性抓取结束
开始BV号为1ET4y1c7iH视频属性的抓取
BV号为1ET4y1c7iH的视频属性抓取结束
开始BV号为18K4y177xW视频属性的抓取
BV号为18K4y177xW的视频属性抓取结束
开始BV号为1qa411A7JF视频属性的抓取
BV号为1qa411A7JF的视频属性抓取结束
开始BV号为1Lh41197Fr视频属性的抓取
BV号为1Lh41197Fr的视频属性抓取结束
开始BV号为1b54y117oM视频属性的抓取
BV号为1b54y117oM的视频属性抓取结束
开始BV号为1dv411k7ox视频属性的抓取
BV号为1dv411k7ox的视频属性抓取结束
开始BV号为1fZ4y1V7Ea视频属性的抓取
BV号为1fZ4y1V7Ea的视频属性抓取结束
开始BV号为1754y1C7wS视频属性的抓取
BV号为1754y1C7wS的视

In [12]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-12-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [14]:
channel_yinyue_VOCALOID·UTAU.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
1月,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90
2月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [15]:
channel_yinyue_VOCALOID·UTAU.to_json('channel_yinyue_VOCALOID·UTAU.json')

In [11]:
channel_yinyue_VOCALOID·UTAU

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1KJ411C7CW,1月,2020-01-24 22:54:38,litterzy,个人认证,36.1万,268,万古生香【2020拜年祭单品】,音乐,VOCALOID·UTAU,合作视频,...,"[洛天依, VOCALOID, 音乐, VOCALOID·UTAU, 万古生香, VOCAL...",04:53,725.1万,35.9万,35.9万,28.5万,5.6万,93568,4.3万,否
1KJ411C7Un,1月,2020-01-24 22:54:38,MitchieM,个人认证,15.3万,26,【Mitchie M】初音未来《买买买》【2020拜年祭单品】,音乐,VOCALOID·UTAU,普通视频,...,"[动感视频, 初音未来, VOCALOID, 音乐, VOCALOID·UTAU, 买买买,...",04:01,320.9万,15.7万,13.5万,12.0万,2.3万,6597,1.5万,否
1KJ411C7Y1,1月,2020-01-24 22:54:38,纳兰寻风,个人认证,19.1万,170,流年如歌【2020拜年祭单品】,音乐,VOCALOID·UTAU,合作视频,...,"[音乐, VOCALOID·UTAU, 流年如歌, 2020拜年祭单品]",04:45,179.6万,7.2万,6.7万,6.5万,9126,6015,2.3万,否
1GJ411J7wG,1月,2020-01-17 17:10:21,JUSF周存,个人认证,44.9万,112,【镜音铃原创曲】请踢断我的肋骨吧【PV付】,音乐,VOCALOID·UTAU,普通视频,...,"[镜音RIN, VOCALOID, 音乐, VOCALOID·UTAU, 镜音铃, JUSF...",04:06,85.1万,3.9万,3.6万,3.6万,6807,4132,2321,否
1tJ411E7Yu,1月,2020-01-17 20:01:58,DECO27_Official,个人认证,28.9万,19,DECO*27 - ポジティブ・パレード feat. 初音未来,音乐,VOCALOID·UTAU,普通视频,...,"[初音未来, VOCALOID, 音乐, VOCALOID·UTAU, DECO*27]",03:04,74.1万,5.6万,5.7万,5.2万,7108,2836,3606,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1p5411879s,8月,2020-08-10 09:10:10,ilem,个人认证,146.3万,72,《苍蝇》 赤羽 原创,音乐,VOCALOID·UTAU,普通视频,...,"[原创歌曲, 音乐, VOCALOID·UTAU, ilem, 赤羽, 五维介质, SYNTHV]",03:20,331.6万,26.0万,19.0万,13.8万,3.2万,17305,1.4万,否
1Ya4y1E7pk,8月,2020-08-02 18:13:44,星のカケラ,无,9734,811,【GUMI】KING【Kanaria】,音乐,VOCALOID·UTAU,普通视频,...,"[GUMI, VOCALOID, 音乐, VOCALOID·UTAU, Kanaria, K...",02:14,142.4万,7.5万,1.7万,8.5万,8731,2123,1333,否
1xD4y127im,8月,2020-08-14 18:00:19,忘川风华录,个人认证,46.2万,22,【赤羽原创】不可道【忘川风华录】,音乐,VOCALOID·UTAU,活动作品,...,"[天文, 中国风, 原创歌曲, 古风, 科普, 音乐, VOCALOID·UTAU, 国风音...",04:19,114.0万,7.4万,6.9万,5.2万,1.2万,8237,1.1万,否
1Bi4y1M7ro,8月,2020-08-27 22:54:21,iliru开头念莉,无,1942,102,【初音ミク】跟笨蛋一樣(da me da ne),音乐,VOCALOID·UTAU,活动作品,...,"[如龙, 初音未来, VOCALOID, 音游, 音乐, VOCALOID·UTAU, 音乐...",01:47,85.1万,4.9万,2.8万,4.1万,6844,2385,841,否
